In [2]:
import os
import json
import csv
import email
from email import policy
from email.parser import BytesParser
from dotenv import load_dotenv
import warnings
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [3]:
# function definitions

def eml_to_json(path_to_eml: str) -> dict:
    """Reads an EML file and converts it to a JSON-like dictionary."""
    with open(path_to_eml, 'rb') as f:
        msg = BytesParser(policy=policy.default).parse(f)

    mail_json = {
        "from": msg["from"],
        "to": msg["to"],
        "subject": msg["subject"],
        "date": msg["date"],
        "body": get_body(msg),
    }
    return mail_json

def get_body(msg):
    """Extracts the body from an email message object."""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                return part.get_content()
    else:
        return msg.get_content()


In [51]:
# load environment variables from .env file
load_dotenv()

# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=512,
)

In [52]:
# Output-Schema definieren
response_schemas = [
    ResponseSchema(
        name="StatusAngebot",
        description="E-Mail enthält eine Angebotsanfrage, 1 wenn ja, 0 wenn nicht, 2 wenn unklar",
        type="Boolean",
    ),
    ResponseSchema(
        name="Universität",
        description="Anfrage von Universitäten oder Studenten, 1 wenn ja, 0 wenn nein, 2 wenn unklar",
        type="Boolean",
    ),
    ResponseSchema(
        name="PhaseCube",
        description="Anfrage enthält das Wort 'PhaseCube', 1 wenn ja, 0 wenn nein, 2 wenn unklar",
        type="Boolean",
    ),
    ResponseSchema(
        name="PhaseTube",
        description="Anfrage enthält das Wort 'PhaseTube', 1 wenn ja, 0 wenn nein, 2 wenn unklar",
        type="Boolean",
    ),
    ResponseSchema(
        name="PhaseDrum",
        description="Anfrage enthält das Wort 'PhaseDrum', 1 wenn ja, 0 wenn nein, 2 wenn unklar",
        type="Boolean",
    )
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
# Prepare prompt
prompt_template = ChatPromptTemplate.from_template(
    """
    Du bist ein Klassifizierer für E-Mails.
    Du erhältst eine E-Mail im JSON-Format.

    Aufgabe:
    - Prüfe, ob es sich um eine Anfrage für ein Angebot handelt.
    - Prüfe, ob die Anfrage von einer Universität oder einem Studenten kommt.
    - Prüfe ob die Mail das Wort "PhaseCube" enthält.
    - Prüfe ob die Mail das Wort "PhaseTube" enthält.
    - Prüfe ob die Mail das Wort "PhaseDrum" enthält.
    - Antworte mit 1 für Ja und 0 für Nein und 2 für Unklar.
    - Antworte ausschließlich im JSON-Format gemäß den Vorgaben.

    Hier sind Beispiele für Mail bodys die nach einem Angebot fragen:
    1. "ich würde ca 5 Kilo von dem PCM RT47 benötigen. Können sie mir bitte ein Angebot zukommen lassen inkl. Lieferung zur Audi AG Ingolstadt?"
    2. "Könnten Sie uns daher bitte ein Angebot für ein Muster sowie für verschiedene Gebindegrößen erstellen?"
    3. "Gerne auch ein Angebot über die 1m3 RT69HC, welche Lieferformen sind hier möglich?"
    4. "Ich freue mich über ein entsprechendes Angebot inklusive Versandkosten."
    5. "Um jedoch eine endgültige Entscheidung treffen zu können, benötige ich die wirtschaftlichen und finanziellen Details zu jedem einzelnen Produkt. Ich bitte Sie daher, mir die Preise für die folgenden Produkte jeweils einzeln mitzuteilen:"
    6. "Please provide the offer for PCM encapsulated materials for the cold storage unit for the parameters below"
    
    Hier sind Beispiele für Mail subjects die nach einem Angebot fragen:
    1. "Anfrage Angebot PCM"
    2. "Anfrage Angebot"
    3. "Anfrage für ein Angebot"
    4. "Bitte um Angebot"
    5. "Angebotserstellung"
    6. "Request for Quotation"
    7. "Quotation Request"
    8. "Request for Quote"
    9. "Quote Request"
    10. "Request for Pricing"

    Hier sind Beispiele für Mails von Universitäten oder Studenten:
    1. "Ich bin Student an der Universität Stuttgart und arbeite derzeit an einem Projekt"
    2. "Ich schreibe meine Masterarbeit an der Technischen Universität München"
    3. "Wir sind eine Forschungsgruppe an der Universität Heidelberg"
    4. "Als Student der RWTH Aachen interessiere ich mich für Ihre Produkte"
    5. "Ich bin Doktorand an der Universität Freiburg und untersuche thermische Energiespeicherung"
    6. "We are a research team from the University of Cambridge"
    7. "I am a graduate student at MIT working on a thesis related to phase change materials"
    8. "Our lab at Stanford University is exploring new applications for PCM"
    9. "As a student at ETH Zurich, I am conducting experiments on thermal storage"
    10. "I am pursuing my PhD at the University of Tokyo and studying advanced materials"
    

    Formatvorgaben:
    {format_instructions}

    Hier ist die E-Mail:
    {mail}
    """
)

In [ ]:
# # Create JSON from EML files
# eml_folder = "..\data\MailsRubitherm"

# json_mails = {}

# for filename in os.listdir(eml_folder):
#         if filename.endswith(".eml"):
#             filepath = os.path.join(eml_folder, filename)
#             mail_json = eml_to_json(filepath)

#             json_mails.update({
#                   filename: {"mail": mail_json}
#             })

# with open("..\\data\\mails.json", "w", encoding="utf-8") as f:
#       json.dump(json_mails, f, indent=2, ensure_ascii=False)

In [ ]:
# Pipeline

def classify_eml_folder(eml_folder: str, json_output: str, csv_output: str):
    results = []

    for filename in os.listdir(eml_folder):
        if filename.endswith(".eml"):
            filepath = os.path.join(eml_folder, filename)
            mail_json = eml_to_json(filepath)

            # fill prompt
            prompt = prompt_template.format(
                format_instructions=format_instructions,
                mail=mail_json
            )

            # request to llm model
            response = llm_model.invoke(prompt)

            try:
                parsed = output_parser.parse(response.content)
            except Exception as e:
                parsed = {"StatusAngebot": None, "Fehler": str(e)}

            results.append({
                "filename": filename,
                "mail": mail_json,
                "klassifikation": parsed
            })

    # Save results as JSON
    with open(json_output, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
 
    # Save results as CSV
    with open(csv_output, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "from", "body", "StatusAngebot", "Universität", "PhaseCube", "PhaseTube", "PhaseDrum"])

        for r in results:
            writer.writerow([
                r["filename"],
                r["mail"].get("from", ""),
                r["mail"].get("body", "").replace("\n", " ").replace("\r", " "),
                r["klassifikation"].get("StatusAngebot", ""),
                r["klassifikation"].get("Universität", ""),
                r["klassifikation"].get("PhaseCube", ""),
                r["klassifikation"].get("PhaseTube", ""),
                r["klassifikation"].get("PhaseDrum", ""),
            ])

    print(f"Fertig. Ergebnisse gespeichert in:\n- JSON: {json_output}\n- CSV:  {csv_output}")

In [55]:
if __name__ == "__main__":
    eml_ordner = "..\data\MailsRubitherm"
    json_datei = "klassifizierte_mails.json"
    csv_datei = "klassifizierte_mails.csv"
    classify_eml_folder(eml_ordner, json_datei, csv_datei)

Fertig. Ergebnisse gespeichert in:
- JSON: klassifizierte_mails.json
- CSV:  klassifizierte_mails.csv
